### About this result

Analyzing the results runnning on IBM quantum platform, the result data is stored in `./pred_outputs/ibmq` with `npy` format.

In [1]:
import os
from itertools import product

import numpy as np

import g_main

general_config = g_main.json_config
ibmq_dir = os.path.join(general_config["predictions_dir"], "ibmq")

In [2]:
shots = 1024
batch_size = 10
num_pt_ptcs = 4
num_bin_data = 25
backend_list = ["ibm_cusco"]
channel_list = ["BB-QCD", "TT-QCD"]

def get_ibmq_output(npy_path):
    result = np.load(npy_path, allow_pickle=True)
    result = result.item()
    return result

rnd_seed= 2
for abbrev, backend in product(channel_list, backend_list):
    print(f"* Channel: {abbrev} | Backend: {backend} | Size: {100}\n")
    print(f"      | SIM  ACC | IBMQ ACC | S/I  ACC | S/I DIFF")
    for Q in [3, 6, 9]:
        # Horizontal splitting line.
        print(f"--------------------------------------------------")
        
        # Read npy file.
        npy_file = (
            f"{backend}-s{shots}-{abbrev}_"
            + f"r{rnd_seed}-p{num_pt_ptcs}_"
            + f"n{num_bin_data}_"
            + f"b{batch_size}-{Q}_1_{Q}"
            + ".npy"
        )
        result = np.load(os.path.join(ibmq_dir, npy_file), allow_pickle=True)
        result = result.item()

        # Prepare true labels and predictions.
        y_true = result["y_true"].reshape(-1).numpy()
        y_penl = result["y_penl"].reshape(-1).numpy()
        y_ibmq = result["y_ibmq"].reshape(-1).numpy()

        # Calculate accuracy and loss (difference).
        SIM__ACC = (y_true == (y_penl > 0)).mean()
        IBMQ_ACC = (y_true == (y_ibmq > 0)).mean()
        S_I__ACC = ((y_penl > 0) == (y_ibmq > 0)).mean()
        S_I_DIFF = np.mean(np.abs(y_penl - y_ibmq))
        print(f"Q = {Q} |   {SIM__ACC:.2f}   |   {IBMQ_ACC:.2f}   |   {S_I__ACC:.2f}   |   {S_I_DIFF:.2f}   ")
    print("\n" + "#"*50 + "\n")

* Channel: BB-QCD | Backend: ibm_cusco | Size: 100

      | SIM  ACC | IBMQ ACC | S/I  ACC | S/I DIFF
--------------------------------------------------
Q = 3 |   0.48   |   0.50   |   0.96   |   0.35   
--------------------------------------------------
Q = 6 |   0.40   |   0.50   |   0.24   |   0.55   
--------------------------------------------------
Q = 9 |   0.52   |   0.50   |   0.98   |   0.10   

##################################################

* Channel: TT-QCD | Backend: ibm_cusco | Size: 100

      | SIM  ACC | IBMQ ACC | S/I  ACC | S/I DIFF
--------------------------------------------------
Q = 3 |   0.59   |   0.50   |   0.85   |   0.46   
--------------------------------------------------
Q = 6 |   0.63   |   0.50   |   0.63   |   0.30   
--------------------------------------------------
Q = 9 |   0.52   |   0.48   |   0.60   |   0.25   

##################################################

